### 1. Extração dos Prospectos dos Fundos

Os prospectos dos fundos (FII, FIDC e FIP) serão baixados da tabela `desafio_kinea.prospecto_fundos.regulamento`.

#### 1.1. Filtrando Prospectos 

In [0]:
# Quantidade Total de Propectos Extraídos
print(spark.read.table("desafio_kinea.prospecto_fundos.regulamento").count())

In [0]:
display(spark.table("desafio_kinea.prospecto_fundos.regulamento"))

In [0]:
display(spark.table("desafio_kinea.prospecto_fundos.regulamento").groupBy("tp_fundo").count())

**Etapa 1**: Filtrando os prospectos emitidos pelos seguintes tipos de fundos: FII, FIDC e FIP.

In [0]:
from pyspark.sql import functions as F
from pyspark.sql import Window

# Carrega a tabela
df = spark.table("desafio_kinea.prospecto_fundos.regulamento")

# Filtra os tp_fundo desejados
df = df.filter(F.col("tp_fundo").isin(["FIDC", "FIP", "F.I.I."]))

print(df.count())

In [0]:
display(df.groupBy("tp_fundo").count())

**Etapa 2:** Excluindo os fundos CANCELADOS cruzando com a base `desafio_kinea.prospecto_fundos.cadastro_175_consolidada`

In [0]:
# Base consolidada de cadastro dos fundos
cadastro = spark.read.table("desafio_kinea.prospecto_fundos.cadastro_175_consolidada")

# Filtra apenas os fundos ativos
f_ativos = cadastro.filter(F.col("sit").isin("Em Funcionamento Normal", "Fase Pré-Operacional")) \
                   .select("cnpj_fundo").distinct()

# Join com o DataFrame filtrado para manter apenas os fundos ativos
df_filtrado = df.join(f_ativos, on="cnpj_fundo", how="inner")

print(df_filtrado.count())

In [0]:
display(df_filtrado)

**Etapa 3:** Filtrandos prospectos com o mesmo CPNJ considerando apenas o mais recente em uma janela < 30 dias. Para janelas > 30 dias serão mantidas por serem consideradas emissões distintas.

In [0]:
from pyspark.sql import functions as F
from pyspark.sql import Window # Funcao para criacao de janelas

# Extrai o timestamp do nome do arquivo

#df_filtrado = df_filtrado.withColumn(
 #   "data_publicacao",
 #  F.to_timestamp(F.substring("nm_arq", 1, 14), "yyyyMMddHHmmss"))

# Define objeto 'janela' por CNPJ ordenada pela data
janela = Window.partitionBy("cnpj_fundo").orderBy("dt_receb")

# Calcula a data anterior
df_filtrado = df_filtrado.withColumn(
    "data_anterior",
    F.lag("dt_receb").over(janela)
)

# Determina inicio de novo grupo (diferença ≥ 30 dias ou é o primeiro doc)
df_filtrado = df_filtrado.withColumn(
    "nova_emissao",
    F.when(F.col("data_anterior").isNull(), 1)
     .when(F.datediff("dt_receb", F.col("data_anterior")) >= 30, 1)
     .otherwise(0)
)

# Soma cumulativa para gerar grupo de emissão
df_filtrado = df_filtrado.withColumn(
    "grupo_emissao",
    F.sum("nova_emissao").over(janela.rowsBetween(Window.unboundedPreceding, 0))
)

# Ordena dentro de cada grupo_emissao e cnpj para pegar o mais recente
janela_grupo = Window.partitionBy("cnpj_fundo", "grupo_emissao").orderBy(F.col("dt_receb").desc())

df_final = df_filtrado.withColumn("ordem", F.row_number().over(janela_grupo)).filter(F.col("ordem") == 1)

print(df_final.count())

In [0]:
display(df_final)

In [0]:
display(df_final.groupBy("tp_fundo").count())

#### 1.2. Baixando Arquivos sem Extensão

Limpando a pasta em caso de alterações nas filtragens

In [0]:
# volume_path = "/Volumes/desafio_kinea/prospecto_fundos/ext-arquivos-prospectos/arquivos-sem-extensao/"
# dbutils.fs.rm(volume_path, recurse=True)  # Remove tudo
# dbutils.fs.mkdirs(volume_path)  # Recria a pasta vazia

Avaliando quais os tipos de extensão

In [0]:
from pyspark.sql.functions import regexp_extract, lower, col

# Expressão regular para extrair a extensão do final do link
# Captura tudo após o último ponto antes de ?, # ou final da string
df_ext = df_final.withColumn(
    "extensao",
    regexp_extract(lower(col("link_arq")), r"\.([a-z0-9]+)(?:\?|#|$)", 1)
)

# Contar quantas vezes cada extensão aparece
ext_counts = df_ext.groupBy("extensao").count().orderBy("count", ascending=False)

ext_counts.show(truncate=False)


Baixando os Arquivos sem extensão

In [0]:
from pyspark.sql.functions import col, lower, trim, regexp_extract
import os
import requests
import mimetypes
from datetime import datetime
from urllib.parse import urlparse

# Caminho para salvar os arquivos
DIR_DOCS = "/Volumes/desafio_kinea/prospecto_fundos/ext-arquivos-prospectos/arquivos-sem-extensao/"
os.makedirs(DIR_DOCS, exist_ok=True)

# Arquivo de log de falhas
LOG_FALHAS = os.path.join(DIR_DOCS, "falhas_download.txt")

# Configuração da sessão HTTP
session = requests.Session()
session.headers.update({
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
})

# Filtrar apenas URLs que NÃO terminam com extensões conhecidas
df_filtrado = (
    df_final.filter(
        (lower(trim(col("tp_doc"))) == "prospec.distrib") &
        col("cnpj_fundo").isNotNull() &
        col("link_arq").isNotNull() &
        ~lower(col("link_arq")).rlike(r'\.(pdf|doc|docx|xls|xlsx|zip|rar|7z)$')
    )
    .select("cnpj_fundo", "tp_fundo", "link_arq", "dt_receb")
)

registros = df_filtrado.collect()

baixados, ignorados, falhas = 0, 0, 0

for i, row in enumerate(registros, 1):
    cnpj = row["cnpj_fundo"].replace(".", "").replace("/", "").replace("-", "")
    tipo = row["tp_fundo"]
    link = row["link_arq"]
    dt_receb = row["dt_receb"]

    # Formatar a data
    try:
        if isinstance(dt_receb, str):
            data_str = datetime.strptime(dt_receb[:10], "%Y-%m-%d").strftime("%Y%m%d")
        else:
            data_str = dt_receb.strftime("%Y%m%d")
    except Exception:
        data_str = "semdata"

    nome_base = f"{tipo}_{cnpj}_{data_str}"

    try:
        # Faz a requisição
        resp = session.get(link, stream=True, timeout=60)
        
        if resp.status_code == 200:
            # Tenta determinar o tipo de conteúdo
            content_type = resp.headers.get('Content-Type', '')
            extensao = mimetypes.guess_extension(content_type.split(';')[0].strip())
            
            # Se não conseguir pela Content-Type, verifica no Content-Disposition
            if not extensao:
                content_disp = resp.headers.get('Content-Disposition', '')
                if 'filename=' in content_disp:
                    filename = content_disp.split('filename=')[1].strip('"\'')
                    extensao = os.path.splitext(filename)[1]
            
            # Se ainda não encontrou extensão, verifica o conteúdo
            if not extensao:
                sample = resp.raw.read(1024)
                resp.raw.seek(0)  # Volta para o início
                
                if sample.startswith(b'%PDF-'):
                    extensao = '.pdf'
                elif sample.startswith(b'\xD0\xCF\x11\xE0'):  # Arquivos DOC antigos
                    extensao = '.doc'
                elif sample.startswith(b'PK\x03\x04'):  # Arquivos Office modernos
                    extensao = '.docx'  # Padrão para Office
                else:
                    extensao = '.bin'  # Extensão genérica
            
            nome_arquivo = f"{nome_base}{extensao}"
            caminho_arquivo = os.path.join(DIR_DOCS, nome_arquivo)
            
            if os.path.exists(caminho_arquivo):
                print(f"[{i}/{len(registros)}] ↪ Já existe: {nome_arquivo}")
                ignorados += 1
                continue
            
            # Salva o arquivo
            with open(caminho_arquivo, 'wb') as f:
                for chunk in resp.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
            
            print(f"[{i}/{len(registros)}] ✓ Baixado: {nome_arquivo}")
            baixados += 1
        else:
            falhas += 1
            with open(LOG_FALHAS, 'a') as f_log:
                f_log.write(f"{link} | HTTP {resp.status_code}\n")
            print(f"[{i}/{len(registros)}] ✗ HTTP {resp.status_code}: {link}")
    
    except Exception as e:
        falhas += 1
        with open(LOG_FALHAS, 'a') as f_log:
            f_log.write(f"{link} | EXCEPTION: {str(e)}\n")
        print(f"[{i}/{len(registros)}] ✗ Erro: {str(e)}")

# Resumo
print("\n========== RESUMO ==========")
print(f"✓ Arquivos baixados: {baixados}")
print(f"↪ Ignorados (já existiam): {ignorados}")
print(f"✗ Falhas no download: {falhas}")
print(f"Total processado: {len(registros)}")

Transferindo os `.pdf` para seus respectivos diretórios a partir da quantidade de páginas do arquivo

In [0]:
!pip install PyPDF2

In [0]:
import os
import shutil
from PyPDF2 import PdfReader

# Diretórios no DBFS
DIR_PDFS = "/Volumes/desafio_kinea/prospecto_fundos/ext-arquivos-prospectos/arquivos-sem-extensao/"
DIR_PDFS_CURTO = "/Volumes/desafio_kinea/prospecto_fundos/ext-arquivos-prospectos/arquivos-pdf-curtos/"
DIR_PDFS_LONGO = "/Volumes/desafio_kinea/prospecto_fundos/ext-arquivos-prospectos/arquivos-pdf/"

# Listar arquivos PDF no diretório original
arquivos = [f for f in os.listdir(DIR_PDFS) if f.lower().endswith(".pdf")]

# Contadores
curtos = 0
longos = 0
transferidos = 0
ignorados = 0
falhas = 0

# Processar arquivos
for i, nome_arquivo in enumerate(arquivos, 1):
    caminho_origem = os.path.join(DIR_PDFS, nome_arquivo)

    try:
        reader = PdfReader(caminho_origem)
        num_paginas = len(reader.pages)

        if num_paginas < 3:
            destino = DIR_PDFS_CURTO
            curtos += 1
        else:
            destino = DIR_PDFS_LONGO
            longos += 1

        caminho_destino = os.path.join(destino, nome_arquivo)

        if os.path.exists(caminho_destino):
            print(f"[{i}/{len(arquivos)}] ↪ Arquivo já existe: {nome_arquivo}")
            ignorados += 1
            continue

        shutil.copyfile(caminho_origem, caminho_destino)
        os.remove(caminho_origem)

        print(f"[{i}/{len(arquivos)}] ✓ Movido para: {destino} ({nome_arquivo})")
        transferidos += 1

    except Exception as e:
        print(f"[{i}/{len(arquivos)}] ✗ Erro ao processar {nome_arquivo}: {e}")
        falhas += 1

# Resumo
print("\n========== RESUMO ==========")
print(f"✓ PDFs longos (>= 3 pág): {longos}")
print(f"↪ PDFs curtos (< 3 pág): {curtos}")
print(f"✓ Arquivos movidos com sucesso: {transferidos}")
print(f"↪ Arquivos já existentes nos diretórios finais: {ignorados}")
print(f"✗ Falhas no processamento: {falhas}")
print(f"Total analisado: {len(arquivos)}")
print("============================")



#### 1.3. Baixando os arquivos .doc

Verificando quantos arquivos .doc tenho no meu `df_final`. Comentário: não há arquivos .docx

In [0]:
from pyspark.sql.functions import col, lower, count

# Contar fundos com links .docx
count_docx = df_final.filter(
    lower(col("link_arq")).like("%.doc")
).count()

print(f"Número de fundos com links .doc: {count_docx}")

Limpando volume antes do processamento (caso haja atualizações nos filtros):

In [0]:
# volume_path = "/Volumes/desafio_kinea/prospecto_fundos/ext-arquivos-prospectos/arquivos-docs/"
# dbutils.fs.rm(volume_path, recurse=True)  # Remove tudo
# dbutils.fs.mkdirs(volume_path)  # Recria a pasta vazia

Filtrando e baixandos os prospectos com link do tipo `.doc`

In [0]:
from pyspark.sql.functions import col, lower, trim

DIR_DOCS = "/Volumes/desafio_kinea/prospecto_fundos/ext-arquivos-prospectos/arquivos-docs/"

df_doc_links = df_final.filter(
    (lower(trim(col("tp_doc"))) == "prospec.distrib") &
    (
        lower(col("link_arq")).like("%.doc") |
        lower(col("link_arq")).like("%.docx")
    ) &
    col("cnpj_fundo").isNotNull() &
    col("link_arq").isNotNull())

registros = df_doc_links.select("cnpj_fundo", "tp_fundo", "link_arq", "dt_receb").collect()

baixados = 0
ignorados = 0
falhas = 0

import os
import requests
from datetime import datetime

for i, row in enumerate(registros, 1):
    cnpj = row["cnpj_fundo"].replace(".", "").replace("/", "").replace("-", "")
    tipo = row["tp_fundo"]
    link = row["link_arq"]
    dt_receb = row["dt_receb"]
    if dt_receb is not None:
        if hasattr(dt_receb, "strftime"):
            data_str = dt_receb.strftime("%Y%m%d")
        else:
            try:
                data_str = datetime.strptime(str(dt_receb), "%Y-%m-%d").strftime("%Y%m%d")
            except Exception:
                data_str = str(dt_receb)
    else:
        data_str = "semdata"

    extensao = ".docx" if link.lower().endswith(".docx") else ".doc"
    nome_arquivo = f"{tipo}_{cnpj}_{data_str}{extensao}"
    caminho_arquivo = os.path.join(DIR_DOCS, nome_arquivo)

    print(f"\n[{i}/{len(registros)}] Verificando: {nome_arquivo}")

    if os.path.exists(caminho_arquivo):
        print(f"  ↪ Já existe. Ignorado.")
        ignorados += 1
        continue

    try:
        resp = requests.get(link, timeout=60)
        if resp.status_code == 200:
            with open(caminho_arquivo, "wb") as f:
                f.write(resp.content)
            print(f"  ✓ Baixado com sucesso.")
            baixados += 1
        else:
            print(f"  ✗ Erro HTTP {resp.status_code}")
            falhas += 1

    except Exception as e:
        print(f"  ✗ Erro ao baixar: {e}")
        falhas += 1

print("\n========== RESUMO ==========")
print(f"✓ Novos arquivos baixados: {baixados}")
print(f"↪ Já existiam e foram ignorados: {ignorados}")
print(f"✗ Falhas no download: {falhas}")
print(f"Total processado: {len(registros)}")
print("============================")

In [0]:
print(spark.read.format("binaryFile").load("/Volumes/desafio_kinea/prospecto_fundos/ext-arquivos-prospectos/arquivos-docs/").count())

Observação: em caso de não funcionamento da azure function, há a possibilidade de conversão dos `.doc` na máquina local. A partir disso, é possível reorganizar os arquivos doc convertidos em pdf a partir da quantidade de páginas.

In [0]:
import os
import shutil
from PyPDF2 import PdfReader

# Diretórios no DBFS
DIR_PDFS = "/Volumes/desafio_kinea/prospecto_fundos/ext-arquivos-prospectos/arquivos-docs-convertidos/"
DIR_PDFS_CURTO = "/Volumes/desafio_kinea/prospecto_fundos/ext-arquivos-prospectos/arquivos-pdf-curtos/"
DIR_PDFS_LONGO = "/Volumes/desafio_kinea/prospecto_fundos/ext-arquivos-prospectos/arquivos-pdf/"

# Listar arquivos PDF no diretório original
arquivos = [f for f in os.listdir(DIR_PDFS) if f.lower().endswith(".pdf")]

# Contadores
curtos = 0
longos = 0
transferidos = 0
ignorados = 0
falhas = 0

# Processar arquivos
for i, nome_arquivo in enumerate(arquivos, 1):
    caminho_origem = os.path.join(DIR_PDFS, nome_arquivo)

    try:
        reader = PdfReader(caminho_origem)
        num_paginas = len(reader.pages)

        if num_paginas < 3:
            destino = DIR_PDFS_CURTO
            curtos += 1
        else:
            destino = DIR_PDFS_LONGO
            longos += 1

        caminho_destino = os.path.join(destino, nome_arquivo)

        if os.path.exists(caminho_destino):
            print(f"[{i}/{len(arquivos)}] ↪ Arquivo já existe: {nome_arquivo}")
            ignorados += 1
            continue

        shutil.copyfile(caminho_origem, caminho_destino)
        os.remove(caminho_origem)

        print(f"[{i}/{len(arquivos)}] ✓ Movido para: {destino} ({nome_arquivo})")
        transferidos += 1

    except Exception as e:
        print(f"[{i}/{len(arquivos)}] ✗ Erro ao processar {nome_arquivo}: {e}")
        falhas += 1

# Resumo
print("\n========== RESUMO ==========")
print(f"✓ PDFs longos (>= 3 pág): {longos}")
print(f"↪ PDFs curtos (< 3 pág): {curtos}")
print(f"✓ Arquivos movidos com sucesso: {transferidos}")
print(f"↪ Arquivos já existentes nos diretórios finais: {ignorados}")
print(f"✗ Falhas no processamento: {falhas}")
print(f"Total analisado: {len(arquivos)}")
print("============================")

#### 1.4. Baixando arquivos `.pdf`

Limpando volume antes do processamento (caso haja atualizações nos filtros):

In [0]:
# volume_path = "/Volumes/desafio_kinea/prospecto_fundos/ext-arquivos-prospectos/arquivos-pdf/"
# dbutils.fs.rm(volume_path, recurse=True)  # Remove tudo
# dbutils.fs.mkdirs(volume_path)  # Recria a pasta vazia

In [0]:
# volume_path = "/Volumes/desafio_kinea/prospecto_fundos/ext-arquivos-prospectos/arquivos-pdf-curtos/"
# dbutils.fs.rm(volume_path, recurse=True)  # Remove tudo
# dbutils.fs.mkdirs(volume_path)  # Recria a pasta vazia

Filtrando os prospectos com link .pdf e separando aqueles com < 3 páginas (prospectos redirecionadores de link)

In [0]:
!pip install PyPDF2

In [0]:
from pyspark.sql.functions import col, lower, trim
import os
import shutil
import requests
from PyPDF2 import PdfReader

DIR_PDFS = "/Volumes/desafio_kinea/prospecto_fundos/ext-arquivos-prospectos/arquivos-pdf/"
DIR_PDFS_CURTO = "/Volumes/desafio_kinea/prospecto_fundos/ext-arquivos-prospectos/arquivos-pdf-curtos/"

os.makedirs(DIR_PDFS_CURTO, exist_ok=True)

df_pdf_links = df_final.filter(
    (lower(trim(col("tp_doc"))) == "prospec.distrib") &
    (lower(col("link_arq")).like("%.pdf")) &
    col("cnpj_fundo").isNotNull() &
    col("link_arq").isNotNull())

registros = df_pdf_links.select("cnpj_fundo", "tp_fundo", "link_arq", "dt_receb").collect()

baixados = 0
ignorados = 0
falhas = 0
curtos = 0

for i, row in enumerate(registros, 1):
    cnpj = row["cnpj_fundo"].replace(".", "").replace("/", "").replace("-", "")
    tipo = row["tp_fundo"]
    link = row["link_arq"]
    dt_receb = row["dt_receb"]
    if dt_receb is not None:
        if hasattr(dt_receb, "strftime"):
            data_str = dt_receb.strftime("%Y%m%d")
        else:
            try:
                # Tenta converter para datetime se for string ou date
                from datetime import datetime
                data_str = datetime.strptime(str(dt_receb), "%Y-%m-%d").strftime("%Y%m%d")
            except Exception:
                data_str = str(dt_receb)
    else:
        data_str = "semdata"

    nome_arquivo = f"{tipo}_{cnpj}_{data_str}.pdf"
    caminho_arquivo = os.path.join(DIR_PDFS, nome_arquivo)

    print(f"\n[{i}/{len(registros)}] Verificando: {nome_arquivo}")

    if os.path.exists(caminho_arquivo):
        print(f"  ↪ Já existe. Ignorado.")
        ignorados += 1
    else:
        try:
            resp = requests.get(link, timeout=60)
            if resp.status_code == 200:
                with open(caminho_arquivo, "wb") as f:
                    f.write(resp.content)
                print(f"  ✓ Baixado com sucesso.")
                baixados += 1
            else:
                print(f"  ✗ Erro HTTP {resp.status_code}")
                falhas += 1
                continue
        except Exception as e:
            print(f"  ✗ Erro ao baixar: {e}")
            falhas += 1
            continue

    try:
        with open(caminho_arquivo, "rb") as f:
            reader = PdfReader(f)
            num_paginas = len(reader.pages)

        if num_paginas < 3:
            destino_curto = os.path.join(DIR_PDFS_CURTO, nome_arquivo)
            shutil.move(caminho_arquivo, destino_curto)
            print(f"  ⚠️ PDF com {num_paginas} página(s), movido para arquivos-pdf-curto.")
            curtos += 1

    except Exception as e:
        print(f"  ✗ Erro ao ler PDF {nome_arquivo}: {e}")

print("\n========== RESUMO ==========")
print(f"✓ Novos arquivos baixados: {baixados}")
print(f"↪ Já existiam e foram ignorados: {ignorados}")
print(f"✗ Falhas no download: {falhas}")
print(f"↪ PDFs com menos de 3 páginas movidos para arquivos-pdf-curto: {curtos}")
print(f"Total processado: {len(registros)}")
print("============================")

Garantindo a transferência de todos os arquivos-pdf-curtos:

In [0]:
import os
import shutil
from PyPDF2 import PdfReader

DIR_PDFS = "/Volumes/desafio_kinea/prospecto_fundos/ext-arquivos-prospectos/arquivos-pdf/"
DIR_PDFS_CURTO = "/Volumes/desafio_kinea/prospecto_fundos/ext-arquivos-prospectos/arquivos-pdf-curtos/"

os.makedirs(DIR_PDFS_CURTO, exist_ok=True)

arquivos = [f for f in os.listdir(DIR_PDFS) if f.lower().endswith(".pdf")]
curtos = 0

for nome_arquivo in arquivos:
    caminho_arquivo = os.path.join(DIR_PDFS, nome_arquivo)
    try:
        with open(caminho_arquivo, "rb") as f:
            reader = PdfReader(f)
            num_paginas = len(reader.pages)
        if num_paginas < 3:
            destino_curto = os.path.join(DIR_PDFS_CURTO, nome_arquivo)
            shutil.move(caminho_arquivo, destino_curto)
            print(f"⚠️ PDF {nome_arquivo} com {num_paginas} página(s), movido para arquivos-pdf-curtos.")
            curtos += 1
    except Exception as e:
        print(f"✗ Erro ao ler PDF {nome_arquivo}: {e}")

print(f"Total de PDFs movidos para arquivos-pdf-curtos: {curtos}")

Excluindo as variáveis duplicadas no diretório pdf-curtos:

In [0]:
import os
from collections import defaultdict

dir_curto = "/Volumes/desafio_kinea/prospecto_fundos/ext-arquivos-prospectos/arquivos-pdf-curtos/"
arquivos = [f for f in os.listdir(dir_curto) if f.lower().endswith(".pdf")]

# Agrupa arquivos por nome base 
base_to_files = defaultdict(list)
for f in arquivos:
    base = f
    base_to_files[base].append(f)

# Identifica duplicados 
duplicados = [files for files in base_to_files.values() if len(files) > 1]

# Remove duplicados, mantendo apenas um arquivo de cada grupo
for files in duplicados:
    # Mantém o primeiro, remove os demais
    for f in files[1:]:
        caminho = os.path.join(dir_curto, f)
        try:
            os.remove(caminho)
        except Exception:
            pass

In [0]:
print(spark.read.format("binaryFile").load("/Volumes/desafio_kinea/prospecto_fundos/ext-arquivos-prospectos/arquivos-pdf/").count())

print(spark.read.format("binaryFile").load("/Volumes/desafio_kinea/prospecto_fundos/ext-arquivos-prospectos/arquivos-pdf-curtos/").count())

#### [Extra] Outras Análises:

Tabelando os fundos com prospectos curtos para Marcelle verificar no fundos.net:

In [0]:
from pyspark.sql.functions import regexp_extract, reverse, split, col, to_date

# Caminho com os PDFs curtos
pdf_curto_path = "/Volumes/desafio_kinea/prospecto_fundos/ext-arquivos-prospectos/arquivos-pdf-curtos/"

# Lê arquivos PDF 
df_files = spark.read.format("binaryFile") \
    .option("pathGlobFilter", "*.pdf") \
    .load(pdf_curto_path)

# extrair informações do nome do arquivo
regex = r"([^/_]+)_([0-9]{14})_([0-9]{8})\.pdf$"

# Extrai apenas o nome do arquivo a partir de _metadata.file_path
df_fundos_curto = df_files.withColumn(
    "filename", reverse(split(col("_metadata.file_path"), "/"))[0]
).withColumn(
    "tp_fundo", regexp_extract("filename", regex, 1)
).withColumn(
    "cnpj_fundo", regexp_extract("filename", regex, 2)
).withColumn(
    "dt_emissao", regexp_extract("filename", regex, 3)
).withColumn(
    "dt_emissao_data", to_date(col("dt_emissao"), "yyyyMMdd")
).select("tp_fundo", "cnpj_fundo", "dt_emissao", "dt_emissao_data", "filename")

# Salva no Unity Catalog
df_fundos_curto.write.mode("overwrite").option("mergeSchema", "true").saveAsTable("desafio_kinea.prospecto_fundos.fundos_pdf_curtos")

# Visualização
display(df_fundos_curto)

Identificando quais fundos armazenados no Volume são da Kinea. Comentário: nenhum.

In [0]:
from pyspark.sql.functions import regexp_extract, regexp_replace, reverse, split

# Caminho dos PDFs
pdf_path = "/Volumes/desafio_kinea/prospecto_fundos/ext-arquivos-prospectos/arquivos-pdf/"

# Lê os arquivos PDF e extrai o CNPJ do nome do arquivo
df_pdfs = (
    spark.read.format("binaryFile").load(pdf_path)
    .withColumn("file_name", reverse(split("_metadata.file_path", "/"))[0])
    .withColumn("clean_name", regexp_replace("file_name", "[^0-9_]", ""))
    .withColumn("cnpj_fundo", regexp_extract("clean_name", "([0-9]{14})", 1))
    .filter("cnpj_fundo IS NOT NULL")
    .select("cnpj_fundo", "_metadata.file_path")
)

# Lê a tabela de cadastro de fundos
df_cadastro = spark.table("desafio_kinea.prospecto_fundos.cadastro")

# Filtra apenas fundos cuja gestora contém "KINEA"
df_kinea = (
    spark.table("desafio_kinea.prospecto_fundos.cadastro")
    .filter("upper(gestor) LIKE '%KINEA%'")
    .withColumn("cnpj_fundo", regexp_replace("cnpj_fundo", "[^0-9]", ""))
    .select("cnpj_fundo").distinct()
)

# Junta os PDFs com os CNPJs dos fundos da Kinea
df_pdfs_kinea = df_pdfs.join(df_kinea, on="cnpj_fundo", how="inner")

# Exibe os arquivos PDF que pertencem a fundos da Kinea
display(df_pdfs_kinea)

Entendendo quais são os fundos da tabela original que são da kinea. Comentário: apenas 28 e possuem 2 páginas redirecionadoras. 

In [0]:
from pyspark.sql.functions import regexp_extract, regexp_replace, reverse, split

# Caminho dos PDFs
pdf_path = "/Volumes/desafio_kinea/prospecto_fundos/ext-arquivos-prospectos/arquivos-pdf/"

# Lê os arquivos PDF e extrai o CNPJ do nome do arquivo
df_pdfs = spark.table("desafio_kinea.prospecto_fundos.regulamento")

# Lê a tabela de cadastro de fundos
df_cadastro = spark.table("desafio_kinea.prospecto_fundos.cadastro")

# Filtra apenas fundos cuja gestora contém "KINEA"
df_kinea = (
    spark.table("desafio_kinea.prospecto_fundos.cadastro")
    .filter("upper(gestor) LIKE '%KINEA%'")
    .select("cnpj_fundo").distinct()
)

# Junta os PDFs com os CNPJs dos fundos da Kinea
df_pdfs_kinea = df_pdfs.join(df_kinea, on="cnpj_fundo", how="inner")

# Exibe os arquivos PDF que pertencem a fundos da Kinea
display(df_pdfs_kinea)

In [0]:
from pyspark.sql.functions import reverse, split

# Caminho dos PDFs
pdf_path = "/Volumes/desafio_kinea/prospecto_fundos/ext-arquivos-prospectos/arquivos-pdf/"

# Lê os arquivos PDF e extrai o tipo de prospecto
df_tipos = (
    spark.read.format("binaryFile").load(pdf_path)
    .withColumn("file_name", reverse(split("_metadata.file_path", "/"))[0])
    .withColumn("tipo_prospecto", split("file_name", "_")[0])
    .groupBy("tipo_prospecto")
    .count()
    .orderBy("count", ascending=False)
)

# Exibe a contagem de tipos
display(df_tipos)
